#### RISK

- 30% Por posicion. Ideal de 10-20%
- 1.5 % max loss

### ESTRATEGIA ENTRADA

1. Ultimas 12 semanas entre un 30-100% de movimiento alcista
2. 2-8 semanas de consolidación estando de resistencia de la MA 10 y 20 y siempre por encima de la MA 50.
3. Brakout vela diaria. SL debajo del minimo de la vela diaria
4. El SL depende del ATR del rango. Por ejemplo un ATR del 5% del rango el SL nunca debe ser mayor del 5%

### ESTRATEGIA SALIDA

1. Vender 1/3 o 1/2 de la posicion 3-5 días despues de la entrada y mover el SL a Breakeaven
2. Mover el SL dependiendo los movimiento del stock con la MA 10 o 20
3. Cerrar cuando el precio cierre por debajo de la MA





In [2]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

In [3]:
df = pd.read_csv ('bitcoin.csv')
df.head ()

,Date,Adj Close,Volume
0,2014-09-17,457.334015,21056800
1,2014-09-18,424.440002,34483200
2,2014-09-19,394.795990,37919700
3,2014-09-20,408.903992,36863600
4,2014-09-21,398.821014,26580100


In [4]:


# Calcular el precio promedio del activo
average_price = df['Adj Close'].mean()

# Calcular la desviación estándar del precio
std_deviation = df['Adj Close'].std()

# Definir el rango en términos de desviaciones estándar (por ejemplo, ±1 desviación estándar)
range_lower = average_price - std_deviation
range_upper = average_price + std_deviation

# Identificar los precios dentro del rango
prices_in_range = df[(df['Adj Close'] >= range_lower) & (df['Adj Close'] <= range_upper)]

print("Precio promedio del activo:", average_price)
print("Desviación estándar del precio:", std_deviation)
print("Rango inferior:", range_lower)
print("Rango superior:", range_upper)
print("Número de precios dentro del rango:", len(prices_in_range))

Precio promedio del activo: 15601.545513446514
Desviación estándar del precio: 17238.391728859835
Rango inferior: -1636.8462154133213
Rango superior: 32839.93724230635
Número de precios dentro del rango: 2854


In [5]:
# Calcular rendimientos diarios
df['Daily_Return'] = df['Adj Close'].pct_change()
average_cambio = df['Daily_Return'].mean()

# Calcular otras estadísticas descriptivas
median_cambio = df['Daily_Return'].median()
std_dev_cambio = df['Daily_Return'].std()
min_cambio = df['Daily_Return'].min()
max_cambio = df['Daily_Return'].max()

# Mostrar los resultados
print("Media de Daily_Return:", average_cambio)
print("Mediana de Daily_Return:", median_cambio)
print("Desviación estándar de Daily_Return:", std_dev_cambio)
print("Mínimo de Daily_Return:", min_cambio)
print("Máximo de Daily_Return:", max_cambio)

Media de Daily_Return: 0.0021348111847822406
Mediana de Daily_Return: 0.0013972858086190865
Desviación estándar de Daily_Return: 0.036855752275668584
Mínimo de Daily_Return: -0.37169538560117077
Máximo de Daily_Return: 0.2524716942763181


In [6]:
df.head ()

,Date,Adj Close,Volume,Daily_Return
0,2014-09-17,457.334015,21056800,NaN
1,2014-09-18,424.440002,34483200,-0.071926
2,2014-09-19,394.795990,37919700,-0.069843
3,2014-09-20,408.903992,36863600,0.035735
4,2014-09-21,398.821014,26580100,-0.024659


buscar periodos de 7 dias que hayan tenido un aumento en el precio del 30 o mayor.


In [7]:

# Calcular retornos acumulados para periodos de 7 días
period_length = 7
start_index = 0

while start_index + period_length <= len(df):
    # Obtener subconjunto de datos para el periodo de 7 días
    subset = df.iloc[start_index:start_index + period_length]
    
    # Calcular la suma acumulada de los retornos durante este periodo
    cumulative_return = subset['Daily_Return'].sum()
    
    # Verificar si la suma acumulada de retornos es >= 0.3
    if cumulative_return >= 0.3:
        # Imprimir el día de inicio y fin de este periodo
        start_date = subset.iloc[0]['Date']
        end_date = subset.iloc[-1]['Date']
        print(f"Periodo con suma de retornos >= 0.3 desde {start_date} hasta {end_date}")
        
        # Reiniciar el inicio del próximo periodo después de este punto
        start_index += period_length
    else:
        # Mover el inicio al siguiente día
        start_index += 1

Periodo con suma de retornos >= 0.3 desde 2015-10-28 hasta 2015-11-03
Periodo con suma de retornos >= 0.3 desde 2016-06-10 hasta 2016-06-16
Periodo con suma de retornos >= 0.3 desde 2017-05-17 hasta 2017-05-23
Periodo con suma de retornos >= 0.3 desde 2017-07-16 hasta 2017-07-22
Periodo con suma de retornos >= 0.3 desde 2017-11-13 hasta 2017-11-19
Periodo con suma de retornos >= 0.3 desde 2017-11-30 hasta 2017-12-06
Periodo con suma de retornos >= 0.3 desde 2017-12-31 hasta 2018-01-06
Periodo con suma de retornos >= 0.3 desde 2019-05-07 hasta 2019-05-13
Periodo con suma de retornos >= 0.3 desde 2019-06-20 hasta 2019-06-26
Periodo con suma de retornos >= 0.3 desde 2021-01-01 hasta 2021-01-07
Periodo con suma de retornos >= 0.3 desde 2021-02-02 hasta 2021-02-08
Periodo con suma de retornos >= 0.3 desde 2023-03-11 hasta 2023-03-17


In [8]:
final = []

def encontrar_periodos_retornos(df, period_length, threshold):
    """
    Encuentra periodos en un DataFrame donde la suma acumulada de los retornos durante un período específico
    supere o iguale un umbral dado.

    Argumentos:
    df (pd.DataFrame): DataFrame que contiene los datos de los retornos diarios.
    period_length (int): Número de días en cada periodo sobre el cual calcular la suma acumulada de los retornos.
    threshold (float): Umbral que determina si la suma acumulada de retornos supera este valor.

    Retorna:
    None (imprime los periodos encontrados que cumplen con el criterio)
    """
    start_index = 0
    
    
    while start_index + period_length <= len(df):
        # Obtener subconjunto de datos para el periodo de días especificado
        subset = df.iloc[start_index:start_index + period_length]
        
        # Calcular la suma acumulada de los retornos durante este periodo
        cumulative_return = subset['Daily_Return'].sum()
        
        # Verificar si la suma acumulada de retornos es >= al umbral
        if cumulative_return >= threshold:
            # Imprimir el día de inicio y fin de este periodo
            start_date = subset.iloc[0]['Date']
            end_date = subset.iloc[-1]['Date']
            print(f"Periodo con suma de retornos >= {threshold} desde {start_date} hasta {end_date}")
            
            final.append(end_date)
            # Reiniciar el inicio del próximo periodo después de este punto
            start_index += period_length
        else:
            # Mover el inicio al siguiente día
            start_index += 1


# Ejemplo de uso:
# Supongamos que df es tu DataFrame y quieres encontrar periodos de 7 días con suma de retornos >= 0.3
encontrar_periodos_retornos(df, period_length=7, threshold=0.3)


Periodo con suma de retornos >= 0.3 desde 2015-10-28 hasta 2015-11-03
Periodo con suma de retornos >= 0.3 desde 2016-06-10 hasta 2016-06-16
Periodo con suma de retornos >= 0.3 desde 2017-05-17 hasta 2017-05-23
Periodo con suma de retornos >= 0.3 desde 2017-07-16 hasta 2017-07-22
Periodo con suma de retornos >= 0.3 desde 2017-11-13 hasta 2017-11-19
Periodo con suma de retornos >= 0.3 desde 2017-11-30 hasta 2017-12-06
Periodo con suma de retornos >= 0.3 desde 2017-12-31 hasta 2018-01-06
Periodo con suma de retornos >= 0.3 desde 2019-05-07 hasta 2019-05-13
Periodo con suma de retornos >= 0.3 desde 2019-06-20 hasta 2019-06-26
Periodo con suma de retornos >= 0.3 desde 2021-01-01 hasta 2021-01-07
Periodo con suma de retornos >= 0.3 desde 2021-02-02 hasta 2021-02-08
Periodo con suma de retornos >= 0.3 desde 2023-03-11 hasta 2023-03-17


In [9]:
print (final)

['2015-11-03', '2016-06-16', '2017-05-23', '2017-07-22', '2017-11-19', '2017-12-06', '2018-01-06', '2019-05-13', '2019-06-26', '2021-01-07', '2021-02-08', '2023-03-17']


In [10]:
import pandas as pd

def calculate_std_deviation(df, target_dates):
    """
    Calcula la desviación estándar de los retornos de los 10 días posteriores a cada fecha objetivo en una lista.

    Argumentos:
    df (pd.DataFrame): DataFrame que contiene los datos de los retornos diarios.
    target_dates (list): Lista de fechas objetivo para las cuales se calculará la desviación estándar de los retornos.

    Retorna:
    dict: Un diccionario donde las claves son las fechas objetivo y los valores son las desviaciones estándar de los retornos.
    """
    # Convertir la columna de fechas a tipo datetime si no está en ese formato
    df['Date'] = pd.to_datetime(df['Date'])
    
    std_deviations = {}

    for target_date in target_dates:
        # Encontrar el índice correspondiente a la fecha objetivo
        index_of_target_date = df[df['Date'] == target_date].index

        if not index_of_target_date.empty:
            # Obtener el índice del primer día y el índice del día décimo posterior
            start_index = index_of_target_date[0]
            end_index = start_index + 10

            # Obtener los retornos de los 10 días posteriores
            returns_subset = df.iloc[start_index:end_index + 1]['Daily_Return']

            # Calcular la desviación estándar de los retornos
            std_deviation = returns_subset.std()

            # Agregar la desviación estándar al diccionario
            std_deviations[target_date] = std_deviation
        else:
            print(f"La fecha {target_date} no se encuentra en el DataFrame.")

    return std_deviations

# Ejemplo de uso:
# Supongamos que 'df' es tu DataFrame y 'final' es la lista de fechas objetivo


# Calcular las desviaciones estándar de los retornos para las fechas en 'final'
result_std_deviations = calculate_std_deviation(df, final)

# Imprimir los resultados
for date, std_dev in result_std_deviations.items():
    print(f"Desviación estándar de los retornos de los 10 días posteriores a {date}: {std_dev}")


Desviación estándar de los retornos de los 10 días posteriores a 2015-11-03: 0.06897649578409508
Desviación estándar de los retornos de los 10 días posteriores a 2016-06-16: 0.06473196368764103
Desviación estándar de los retornos de los 10 días posteriores a 2017-05-23: 0.054108672892138346
Desviación estándar de los retornos de los 10 días posteriores a 2017-07-22: 0.04449723092330159
Desviación estándar de los retornos de los 10 días posteriores a 2017-11-19: 0.03113743621731062
Desviación estándar de los retornos de los 10 días posteriores a 2017-12-06: 0.10793575065923895
Desviación estándar de los retornos de los 10 días posteriores a 2018-01-06: 0.06487211751723111
Desviación estándar de los retornos de los 10 días posteriores a 2019-05-13: 0.06263290391200671
Desviación estándar de los retornos de los 10 días posteriores a 2019-06-26: 0.08380190995651571
Desviación estándar de los retornos de los 10 días posteriores a 2021-01-07: 0.05699527131722684
Desviación estándar de los re

In [11]:
final = []

def encontrar_periodos_retornos(df, periodo, retornos):
   
    inicio_dia = 0
    
    while inicio_dia + periodo <= len(df):
        
        subset = df.iloc[inicio_dia:inicio_dia + periodo]
        
        
        cum_retorno = subset['Daily_Return'].sum()
        
        
        if cum_retorno >= retornos:
            
            start_date = subset.iloc[0]['Date']
            end_date = subset.iloc[-1]['Date']
            print(f"Periodo con suma de retornos >= {retornos} desde {start_date} hasta {end_date}")
            
            final.append(end_date)
            
            inicio_dia += periodo
        else:
            
            inicio_dia += 1


encontrar_periodos_retornos(df, periodo=5, retornos=0.2)

Periodo con suma de retornos >= 0.2 desde 2014-11-08 00:00:00 hasta 2014-11-12 00:00:00
Periodo con suma de retornos >= 0.2 desde 2015-01-15 00:00:00 hasta 2015-01-19 00:00:00
Periodo con suma de retornos >= 0.2 desde 2015-10-30 00:00:00 hasta 2015-11-03 00:00:00
Periodo con suma de retornos >= 0.2 desde 2016-06-12 00:00:00 hasta 2016-06-16 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-05-19 00:00:00 hasta 2017-05-23 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-07-16 00:00:00 hasta 2017-07-20 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-08-03 00:00:00 hasta 2017-08-07 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-08-10 00:00:00 hasta 2017-08-14 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-09-15 00:00:00 hasta 2017-09-19 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-10-08 00:00:00 hasta 2017-10-12 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-10-29 00:00:00 hasta 2017-11-02 00:00:00
Periodo con suma de retornos >= 

In [12]:
final = []

def encontrar_periodos_retornos(df, periodo, retornos):
    """
    Función para encontrar periodos en los que la suma de los retornos diarios alcanza un valor objetivo.

    Args:
    - df (DataFrame): DataFrame que contiene la serie de precios con columnas 'Date' y 'Daily_Return'.
    - periodo (int): Número máximo de días para considerar en cada periodo.
    - retornos (float): Valor objetivo de suma de retornos diarios.

    """
    inicio_dia = 0

    while inicio_dia < len(df):
        cum_retorno = 0
        for i in range(periodo):
            if inicio_dia + i < len(df):
                cum_retorno += df.iloc[inicio_dia + i]['Daily_Return']
                if cum_retorno >= retornos:
                    start_date = df.iloc[inicio_dia]['Date']
                    end_date = df.iloc[inicio_dia + i]['Date']
                    print(f"Periodo con suma de retornos >= {retornos} desde {start_date} hasta {end_date}")
                    final.append(end_date)
                    inicio_dia += i + 1
                    break
        else:
            inicio_dia += 1

# Llama a la función encontrar_periodos_retornos con tu DataFrame como argumento
encontrar_periodos_retornos(df, periodo=5, retornos=0.2)


Periodo con suma de retornos >= 0.2 desde 2014-11-08 00:00:00 hasta 2014-11-12 00:00:00
Periodo con suma de retornos >= 0.2 desde 2015-01-15 00:00:00 hasta 2015-01-19 00:00:00
Periodo con suma de retornos >= 0.2 desde 2015-10-30 00:00:00 hasta 2015-11-03 00:00:00
Periodo con suma de retornos >= 0.2 desde 2016-06-10 00:00:00 hasta 2016-06-13 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-05-19 00:00:00 hasta 2017-05-23 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-07-16 00:00:00 hasta 2017-07-20 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-08-03 00:00:00 hasta 2017-08-07 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-08-10 00:00:00 hasta 2017-08-13 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-09-15 00:00:00 hasta 2017-09-18 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-10-08 00:00:00 hasta 2017-10-12 00:00:00
Periodo con suma de retornos >= 0.2 desde 2017-10-29 00:00:00 hasta 2017-11-02 00:00:00
Periodo con suma de retornos >= 

In [19]:
final = []

def encontrar_periodos_retornos(df, periodo, retornos):
    """
    Función para encontrar periodos en los que la suma de los retornos diarios alcanza un valor objetivo.

    Args:
    - df (DataFrame): DataFrame que contiene la serie de precios con columnas 'Date' y 'Daily_Return'.
    - periodo (int): Número máximo de días para considerar en cada periodo.
    - retornos (float): Valor objetivo de suma de retornos diarios.
    """
    inicio_dia = 0

    while inicio_dia < len(df):
        cum_retorno = 0
        for i in range(periodo):
            if inicio_dia + i < len(df):
                cum_retorno += df.iloc[inicio_dia + i]['Daily_Return']
                if cum_retorno >= retornos:
                    start_date = df.iloc[inicio_dia]['Date'].strftime('%Y-%m-%d')
                    end_date = df.iloc[inicio_dia + i]['Date'].strftime('%Y-%m-%d')
                    print(f"Periodo con suma de retornos >= {retornos} desde {start_date} hasta {end_date}")
                    final.append(end_date)
                    inicio_dia += i + 1
                    break
        else:
            inicio_dia += 1

# Llama a la función encontrar_periodos_retornos con tu DataFrame como argumento
encontrar_periodos_retornos(df, periodo=90, retornos=1)


Periodo con suma de retornos >= 1 desde 2017-07-17 hasta 2017-09-01
Periodo con suma de retornos >= 1 desde 2017-10-04 hasta 2017-12-03
Periodo con suma de retornos >= 1 desde 2020-11-08 hasta 2021-01-07


In [21]:
final = []

def encontrar_periodos_retornos(df, periodo, retornos):
    """
    Función para encontrar periodos en los que la suma de los retornos diarios alcanza un valor objetivo.

    Args:
    - df (DataFrame): DataFrame que contiene la serie de precios con columnas 'Date' y 'Daily_Return'.
    - periodo (int): Número máximo de días para considerar en cada periodo.
    - retornos (float): Valor objetivo de suma de retornos diarios.
    """
    inicio_dia = 0

    while inicio_dia < len(df):
        cum_retorno = 0
        for i in range(periodo):
            if inicio_dia + i < len(df):
                cum_retorno += df.iloc[inicio_dia + i]['Daily_Return']
                if cum_retorno >= retornos:
                    start_date = df.iloc[inicio_dia]['Date'].strftime('%Y-%m-%d')
                    end_date = df.iloc[inicio_dia + i]['Date'].strftime('%Y-%m-%d')
                    total_dias = i + 1
                    print(f"Periodo con suma de retornos >= {retornos} desde {start_date} hasta {end_date} en {total_dias} días")
                    final.append(end_date)
                    inicio_dia += i + 1
                    break
        else:
            inicio_dia += 1

# Llama a la función encontrar_periodos_retornos con tu DataFrame como argumento
encontrar_periodos_retornos(df, periodo=95, retornos=1)


Periodo con suma de retornos >= 1 desde 2017-03-09 hasta 2017-06-10 en 94 días
Periodo con suma de retornos >= 1 desde 2017-07-11 hasta 2017-10-13 en 95 días
Periodo con suma de retornos >= 1 desde 2017-10-14 hasta 2017-12-07 en 55 días
Periodo con suma de retornos >= 1 desde 2019-03-20 hasta 2019-06-22 en 95 días
Periodo con suma de retornos >= 1 desde 2020-09-27 hasta 2020-12-30 en 95 días
